In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import itertools
import random

In [2]:
try:

    # Create a new model
    m = gp.Model("2.17")

    # Create variables
    c = np.array([1, 2, 3])
    A = np.array([ [1, 2, 3], [1, 1, 0] ])
    b = np.array([4, 1])
    x_prev = np.array([1,2,3])
    x = m.addMVar(3, lb=0.0)
    m.setObjective(c @ x)
    m.addConstr(A@x == b)

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print('%s %g' % (v.VarName, v.X))

    print('Obj: %g' % m.ObjVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-11
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x1990d0aa
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  4.000000000e+00
C0 1
C1 0
C2 1
Obj: 4


In [7]:
def min_Q_(c: dict, W: dict, h: dict, T: dict, G: dict, g: dict, x_prev: dict, k: int, t: int) -> tuple:

    """
    This function minimizes the function `c[t][k] @ x_t + phi` subject to the following constraints:

    * `W[t][k] @ x_t == h[t][k] - T[t-1][k] @ x_prev`
    * `- G[t+1] @ x_t + phi >= g[t+1]`

    Parameters TODO: Change back to dicts
    ----------
    c : numpy.ndarray
        A 2D array of cost coefficients.
    W : numpy.ndarray
        A 2D array of transition matrices.
    h : numpy.ndarray
        A 1D array of holding costs.
    T : numpy.ndarray
        A 2D array of transfer matrices.
    G : numpy.ndarray
        A 2D array of constraint matrices.
    g : numpy.ndarray
        A 1D array of constraint bounds.
    x_prev : numpy.ndarray
        A 1D array of previous state values.
    k : int
        A scenario index.
    t : int
        A time index.

    Returns
    -------
    x_t_val : numpy.ndarray
        A 1D array of optimal state values.
    obj_val : float
        The optimal objective value.
    pi : float
        The dual value of the first constraint.
    rho : float
        The dual value of the second constraint.
    """

    m = gp.Model("Q_")
    
    x_t = m.addMVar(shape = len(c[t][k]))#, lb=0.0)
    phi = m.addVar()
    
    m.setObjective(c[t][k] @ x_t + phi)
    
    sequence_constraint = m.addConstr(W[t][k] @ x_t == h[t][k] - T[t-1][k] @ x_prev, name = "sequence_constraint")
    cut_constraint = m.addConstr(- G[t+1] @ x_t + phi >= g[t+1] , name = "cut_constraint") #*np.ones_like(g[t+1])
    #print(G[t+1])
    #print(g[t+1])
    m.update()
    #print(f'RHS: {cut_constraint.getAttr("RHS")}')
        
    # Solve
    m.setParam( 'OutputFlag', False )
    m.optimize()
    
    # Primal and objective value
    x_t_val = x_t.getAttr("x")
    obj_val = m.getAttr("ObjVal")
    
    # Duals
    pi = sequence_constraint.getAttr("Pi")
    rho = cut_constraint.getAttr("Pi")

    # Return results
    return x_t_val, obj_val, pi, rho


In [10]:
# Testing
t = 1
k = 0
c = {1: {}}
W = {1: {}}
h = {1: {}}
T = {0: {}}
G = {}
g = {}
c[1][0] = np.array([1, 2, 3])
W[1][0] = np.array([ [1, 2, -4], [1, 1, 0] ])
h[1][0] = np.array([4, 1])
T[0][0] = np.array([ [2,1, 3], [1, 1, 0] ])
x_prev = np.array([1,0,1])
G[2] = np.zeros(shape = (1, len(c[t][0])))
g[2] = np.array([0])
x,Q,pi,rho = min_Q_(c,W,h,T, G,g, x_prev,k,t)

In [11]:
i = 1
t = 1
h = {1: {}}
T = {0: {}}
G = {}
g = {}
q = {}
p = {}
pi = {1:{}}
rho = {1:{}}
c[1][0] = np.array([1, 2, 3])
W[1][0] = np.array([ [1, 2, -4], [1, 1, 0] ])
h[1][0] = np.array([4, 1])
h[1][1] = np.array([2, 1])
h[1][2] = np.array([1, 1])
h[1][3] = np.array([1, 1])
T[0][0] = np.array([ [2,1, 3], [1, 1, 0] ])
T[0][1] = np.array([ [2,1, 3], [1, 1, 0] ])
T[0][2] = np.array([ [2,1, 3], [1, 1, 0] ])
T[0][3] = np.array([ [2,1, 3], [1, 1, 0] ])
x_prev = np.array([1,0,1])
G[1] = np.zeros(shape = (1, len(c[t][0])))
g[1] = np.array([0])
g[2] = np.array([1])

q[1] = 4
p[1] = np.array([0.1,0.2,0.3,0.4])
pi[1][1] = np.array([ [1,1], [1,3], [2,3], [1,2]])
rho[1][1] = np.array([2,3,4,5])

beta = - sum(p[t][j]*np.matmul(pi[i][t][j],T[t-1][j]) for j in range(q[t]))
print(beta)
alpha = sum(p[t][j] * (np.matmul(pi[i][t][j],h[t][j]) + rho[i][t][j]*g[t+1]) for j in range(q[t]))
print(alpha)
print(G)
print(g)
G[t] = np.vstack((G[t],beta))
g[t] = np.vstack((g[t],alpha))
print(G)
print(g)
#x,Q,pi,rho = min_Q_(c,W,h,T, G,g, x_prev,k,t)

[-5.  -3.7 -3.9]
[8.2]
{1: array([[0., 0., 0.]])}
{1: array([0]), 2: array([1])}
{1: array([[ 0. ,  0. ,  0. ],
       [-5. , -3.7, -3.9]])}
{1: array([[0. ],
       [8.2]]), 2: array([1])}


In [ ]:
def calculate_new_cut(q: dict, p: dict, h: dict[int, dict], T: dict[int, dict], 
                      pi: dict[int, dict], rho: dict[int, dict], g: dict, i: int, t: int, scenario: int) -> tuple:
    """
    Calculates the new cut, i.e. beta and alpha based on the input dicts and dual variables.
    
    Args:
    - q: A dictionary representing the number of elements for each time period.
    - p: A dictionary representing the probabilities of choosing an element for each time period.
    - h: A dictionary of dictionaries representing the values for each element at each time period.
    - T: A dictionary of dictionaries representing the transition matrix for each element at each time period.
    - pi: A dictionary of dictionaries of dictionaries representing the transition probabilities of each element at each time period.
    - rho: A dictionary of dictionaries of dictionaries representing the values of each constraint at each time period.
    - g: A dictionary representing the objective function values for each time period.
    
    Returns:
    A tuple containing the values of beta and alpha.
    """
    beta = - sum(p[t][j] * np.matmul(pi[i][t][scenario][j],T[t-1][j]) for j in range(q[t]))
    alpha = sum(p[t][j] * (np.matmul(pi[i][t][scenario][j],h[t][j]) + np.dot(rho[i][t][scenario][j],g[t+1])) for j in range(q[t]))
    return beta, alpha

In [46]:
def add_cut(G_t: np.array, g_t: np.array, beta: np.array, alpha: np.array) -> tuple[np.array, np.array]:
    assert G_t.shape[0] == g_t.shape[0]
    
    G_t = np.vstack((G_t,beta))
    g_t = np.hstack((g_t,alpha))

    assert G_t.shape[0] == g_t.shape[0]

    hstacked = np.hstack((G_t,g_t[:,None]))
    hstacked_unique = np.unique(hstacked, axis=0)
    G_t = hstacked_unique[:, :-1]
    g_t = hstacked_unique[:, -1]
    return G_t, g_t


In [55]:
def remove_cut(G_t: np.ndarray, g_t: np.ndarray, beta: np.ndarray, alpha: np.ndarray) -> tuple[np.ndarray, np.ndarray]:

    assert G_t.shape[0] == g_t.shape[0]

    hstacked = np.hstack((G_t,g_t[:,None]))
    hstacked_unique = np.unique(hstacked, axis=0)

    plane = np.concatenate((beta, alpha))
    hstacked_removed = hstacked_unique[(hstacked_unique != plane).any(axis=1)]

    G_t = hstacked_removed[:, :-1]
    g_t = hstacked_removed[:, -1]
    return G_t, g_t

In [54]:
G_t = np.array([[0,1,1],[1,1,0]])
g_t = np.array([2,3])
beta = np.array([0,1,1])
alpha = np.array([2])
G_t2, g_t2 = remove_cut(G_t, g_t, beta, alpha)
G_t2, g_t2

(array([[1, 1, 0]]), array([3]))

In [15]:
def SDDP(t_max: int, i_max: int, S: dict, c: dict, W: dict, h: dict, T: dict, phi_: np.array, no_of_samples: int, x_0: np.array) -> tuple:
    i = 0
    x = {}
    Q_ = {}
    pi = {}
    rho = {}
    
    G = {}
    g = {}
    
    v_ = {}
    
    # Initialize cut approximations
    for t in range(2,t_max+1):
        G[t] = np.zeros(shape = (1, len(c[t-1][0])))
        g[t] = np.array([phi_[t]])        
    G[t_max+1] = np.zeros(shape = (1, len(c[t_max][0])))
    g[t_max+1] = np.array([0])
    
    
    while i < i_max:
        # Initialize x, Q
        x[i] = {}
        Q_[i] = {}
        pi[i] = {}
        rho[i] = {}
        
        x[i][0] = {}
        for scenario in list(S.keys()):
            x[i][0][scenario] = x_0
            
        for t in range(1,t_max+1):
            x[i][t] = {}
            Q_[i][t] = {}
            pi[i][t] = {}
            rho[i][t] = {}
        
        # Forward pass
        print(f"Forward pass: {i}")
        sampled_scenarios = random.sample(list(S.keys()), no_of_samples)
        for t in range(1,t_max+1):
            for scenario in sampled_scenarios:
                #print(f"t: {t}, scenario: {scenario}")
                k = S[scenario][t]
                x[i][t][scenario], Q_[i][t][scenario], pi_temp, rho_temp = min_Q_(c,W,h,T, G,g, x[i][t-1][scenario],k,t) # Seems good
        
        # calculate upper bound + variances (TODO)
        
        
        # Backward pass
        print(f"Backward pass: {i}")
        for t in reversed(range(1,t_max+1)):
            for scenario in sampled_scenarios:
                
                for j in range(q[t]):
                    #print(f"t: {t}, scenario: {scenario}, j: {j}")
                    x[i][t][scenario], Q_[i][t][scenario], pi[i][t][j], rho[i][t][j] = min_Q_(c,W,h,T,G,g,x[i][t-1][scenario],j,t) # j seems good
                    #print(f"rho[{i}][{t}][{j}]: {rho[i][t][j]}")
                
                # Calculate new cut (beta, alpha)
                beta, alpha = calculate_new_cut(q,p,h,T,pi,rho,g)

                #print(f"g[{t+1}]: {g[t+1]}")
                #print(f"beta: {beta}")
                #print(f"alpha: {alpha}")
                
                # Add new cut (beta, alpha) to G, g:
                if t != 1:
                    G[t] = np.vstack((G[t],beta))
                    g[t] = np.hstack((g[t],alpha))
                    #print(f"g[{t}]: {g[t]}")
                    
        v_[i] = Q_[i][1][scenario]
        print(f"v_[{i}]: {v_[i]}")
        i = i + 1
    return x, Q_

In [56]:
def BL_SDDP(t_max: int, n_max: int, i_max: int, z_max: int, S: dict[int, tuple[int]], c: dict, W: dict, h: dict, T: dict, 
            phi_: np.array, no_of_samples: int, batch_size: int, x_0: np.array) -> tuple:
    
    assert batch_size <= no_of_samples, "Batch size larger than number of samples per stage"

    i = 0
    x = {}
    Q_ = {}
    pi = {}
    rho = {}
    
    G = {}
    g = {}
    
    v_ = {}

    memory = {}

    sampled_scenarios = {}
    
    # Initialize cut approximations
    for t in range(2,t_max+1):
        G[t] = np.zeros(shape = (1, len(c[t-1][0])))
        g[t] = np.array([phi_[t]])        
    G[t_max+1] = np.zeros(shape = (1, len(c[t_max][0])))
    g[t_max+1] = np.array([0])
    
     # Initialize dicts
    for i in range(0, i_max):
        x[i] = {}
        Q_[i] = {}
        pi[i] = {}
        rho[i] = {}
        
        x[i][0] = {}
        for scenario in list(S.keys()):
            x[i][0][scenario] = x_0
        
        for t in range(1,t_max+1):
            x[i][t] = {}
            Q_[i][t] = {}
            pi[i][t] = {}
            rho[i][t] = {}
            

        for t in range(1,t_max+1):
            memory[t] = {}



    i = 1
    while i < i_max:
        l = 0
        z = 0
        while z < z_max:

            l_init = l

            # Sample no_of_samples scenarios
            sampled_scenarios = random.sample(list(S.keys()), no_of_samples)
            
            ## FORWARD PASS

            # Compute actions (forward)
            for t in range(1,t_max+1):
                for scenario in sampled_scenarios:
                    
                    k = S[scenario][t]
                    x[i][t][scenario], Q_[i][t][scenario], pi_temp, rho_temp = min_Q_(c,W,h,T, G,g, x[i][t-1][scenario],k,t)
        
                    # Add trial action to memory
                    memory[t][(i,scenario)] = {"action": x[i][t][scenario]}
                    l = l + 1

            l = l_init
            ## BACKWARD PASS
            
            for t in reversed(range(1,t_max+1)):
                for scenario in sampled_scenarios:
                    
                    # Compute duals (pi, rho) for the scenario
                    for j in range(q[t]):
                        x_temp, Q_temp, pi[i][t][scenario][j], rho[i][t][scenario][j] = min_Q_(c,W,h,T,G,g,x[i][t-1][scenario],j,t)
                    
                    # Calculate new cut (beta, alpha)
                    beta, alpha = calculate_new_cut(q,p,h,T,pi,rho,g,i,t,scenario)

                    # Add new cut (beta, alpha) to G, g:
                    if t != 1:
                        G[t], g[t] = add_cut(G[t], g[t], beta, alpha)

                    # Store cut
                    memory[t][(i,scenario)]["beta"] = beta
                    memory[t][(i,scenario)]["alpha"] = alpha
                    l = l + 1

            # Increase z
            z = z + no_of_samples
            assert l == z, f"l = {l}, z = {z}"
            
        # Select batch keys
        batch_keys = {}
        for t in range(1, t_max+1):

            # Select batch_size number of keys from the memory for stage t
            batch_keys[t] = random.sample(list(memory[t]), batch_size)

            # Remove the K cuts belonging to the K selected actions
            for batch_key in batch_keys[t]:
                beta = memory[t][batch_key]["beta"]
                alpha = memory[t][batch_key]["alpha"]
                G[t], g[t] = remove_cut(G[t], g[t], beta, alpha) # TODO: Implement removal of cut
    
        
        # TODO: Backward pass around the K trial actions
            #  Store K new cuts
        i = i + 1
        



In [45]:
a = np.zeros(shape=(2,3))
b = np.array([1,1])
stacked = np.hstack((a,b[:,None]))
stacked_unique = np.unique(stacked, axis=0)
a_u = stacked_unique[:, :-1]
b_u = stacked_unique[:, -1]
a_u, b_u


(array([[0., 0., 0.]]), array([1.]))

In [377]:
## INPUT
t_max = 12

# Uncertainty
q = {t: 2 for t in range(1,t_max+1)}
p = {t: np.array([0.45,0.55]) for t in range(1,t_max+1)}

list_q = [range(q[t]) for t in range(1,t_max+1)]
scenarios_raw = [w for w in itertools.product(range(1),*list_q)]
S = {i: scenarios_raw[i] for i in range(len(scenarios_raw))}

# Parameters
from_grid_max = 25
to_grid_max = 25
soc_max = 100

p_from_grid = {t: {} for t in range(1,t_max+1)}
p_to_grid = {t: {} for t in range(1,t_max+1)}
gen_less_demand = {t: {} for t in range(1,t_max+1)}

for t in range(1, 6):
    p_from_grid[t] = {0: 0.2, 1: 0.4}
    p_to_grid[t] = {0: 0.1, 1: 0.2} 
    gen_less_demand[t] = {0: 0, 1: 10}

for t in range(6, t_max+1):
    p_from_grid[t] = {0: 0.4, 1: 0.6}
    p_to_grid[t] = {0: 0.2, 1: 0.4}
    gen_less_demand[t] = {0: -20, 1: -5}

# Initial condition
x_0 = np.array([0,0,10,0,0,0])

# Generate input represenation
c = {}
W = {}
h = {}
T = {}
for t in range(1,t_max+1):
    c[t] = {}
    W[t] = {}
    h[t] = {}
    T[t-1] = {}
    for j in range(q[t]):
        c[t][j] = np.array([p_from_grid[t][j], -p_to_grid[t][j], 0, 0, 0, 0])
        W[t][j] = np.array([ [-1, 1, 1, 0, 0, 0], [1, 0, 0, 1, 0, 0], [0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 1] ])
        h[t][j] = np.array([gen_less_demand[t][j], from_grid_max, to_grid_max, soc_max])
        T[t-1][j] = np.array([ [0, 0, -1, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0] ])


# Algorithm parameters
i_max = 20
no_of_samples = 3
phi_list = [-20 for t in range(1, t_max+1)]
phi_ = np.array([0,*phi_list])

x,Q = SDDP(t_max,i_max,S,c,W,h,T,phi_,no_of_samples, x_0)

Forward pass: 0
Backward pass: 0
v_[0]: -2.269251090813346
Forward pass: 1
Backward pass: 1
v_[1]: 7.10000000000002
Forward pass: 2
Backward pass: 2
v_[2]: 8.248952999040313
Forward pass: 3
Backward pass: 3
v_[3]: 8.713309141906855
Forward pass: 4
Backward pass: 4
v_[4]: 8.713309141906855
Forward pass: 5
Backward pass: 5
v_[5]: 8.89144156250001
Forward pass: 6
Backward pass: 6
v_[6]: 8.89144156250001
Forward pass: 7
Backward pass: 7
v_[7]: 8.89144156250001
Forward pass: 8
Backward pass: 8
v_[8]: 8.89144156250001
Forward pass: 9
Backward pass: 9
v_[9]: 8.89144156250001
Forward pass: 10
Backward pass: 10
v_[10]: 8.89144156250001
Forward pass: 11
Backward pass: 11
v_[11]: 8.89144156250001
Forward pass: 12
Backward pass: 12
v_[12]: 8.89144156250001
Forward pass: 13
Backward pass: 13
v_[13]: 8.89144156250001
Forward pass: 14
Backward pass: 14
v_[14]: 8.89144156250001
Forward pass: 15
Backward pass: 15
v_[15]: 8.89144156250001
Forward pass: 16
Backward pass: 16
v_[16]: 8.89144156250001
Forwa

In [382]:
x[19][10]

{1628: array([ 0.  ,  6.75, 23.5 , 25.  , 18.25, 76.5 ]),
 3667: array([ 0. ,  0. , 20.5, 25. , 25. , 79.5]),
 256: array([ 0.,  0., 22., 25., 25., 78.])}

In [143]:
q = {1: 2, 2: 2, 3: 2}
S = [w for w in itertools.product(range(1),range(q[1]),range(q[2]),range(q[3]))]
scenarios = {i: S[i] for i in range(len(S))}
print(scenarios)

[(0, 0, 0, 0), (0, 0, 0, 1), (0, 0, 1, 0), (0, 0, 1, 1), (0, 1, 0, 0), (0, 1, 0, 1), (0, 1, 1, 0), (0, 1, 1, 1)]
{0: (0, 0, 0, 0), 1: (0, 0, 0, 1), 2: (0, 0, 1, 0), 3: (0, 0, 1, 1), 4: (0, 1, 0, 0), 5: (0, 1, 0, 1), 6: (0, 1, 1, 0), 7: (0, 1, 1, 1)}


In [ ]:
#Q[n,t,s_t,a_t]
# Initialize #Q[0,t,s_t,a_t]
# for n = 1,...,N
# Data:

# def SDDP(N,T,S,A, Q_0):
#     for n in range(1,N+1):
#         # Forward pass (initialize s1)
#         s_t = s_1
#         for t in range(1,T+1):
#             a_tn = minimize(Q[n-1,t],s_t,A_t)
#             s_t = transition(a_tn)
#         # Backward pass
#         for t in reversed(range(1,T+1)):
#             Q[n,t-1] = U[t,a_tn,Q[n,t]]
#     return Q